In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import statistics
import math

In [65]:
def longest_streak(url):
    ##print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    return_value = 0
    try:
        spans = soup.find('div', {'id':'meta'}).find('div', {'data-template':'Partials/Teams/Summary'}).find_all('span')
        spans.pop(-1)
        spans.pop(-1)
        team_year = ""
        games_played = 0
        for span in spans:
            team_year += span.get_text()
            team_year += " "
        print(team_year)
        res = soup.find('div', {'id':'timeline_results'}).find_all('ul')
        winpct_lst = []
        date_lst = []
        for ul in res:
            li_lst = ul.find_all('li', {'class':'result'})
            for li in li_lst:
                span = li.find('a')
                record = span.get_text()
                date = str(span.get_text()).split(' @')
                date = date[0]
                date = date.split('. ')
                date = date[1]
                if ',' in date:
                    date = date.split(',')
                    date = date[0]
                date_lst.append(str(date))
                record = record.split('(')
                record = record[1]
                record = record.split(')')
                record = record[0]
                temp = record.split('-')
                wins = int(temp[0])
                losses = int(temp[1])
                games = wins + losses
                total_games = games
                winpct = float(wins) / float(games)
                winpct_lst.append(winpct)
        #print(winpct_lst)
        #print(date_lst)
        max_count = 0
        curr_count = 0
        index = 0
        start = 0
        end = total_games
        for elt in winpct_lst:
           # print('======================================')
           # print("Curr: " + str(curr_count))
            temp = round((100 * elt), 2)
            if temp > 85:
                if index == 81:
                    e = index
                    temp = max_count
                #    print("Max: " + str(max_count))
                    max_count = max(max_count, curr_count)
                #    print("Max: " + str(max_count))
                    if temp != max_count:
                        start = s
                        end = e
                if curr_count == 0:
                    s = index
                curr_count += 1
            else:
                e = index
                temp = max_count
               # print("Max: " + str(max_count))
                max_count = max(max_count, curr_count)
               # print("Max: " + str(max_count))
                if temp != max_count:
                    start = s
                    end = e
                curr_count = 0
            index += 1
        #print(start)
        #print(end)
        #print(max_count)
        lst1 = date_lst[start:end]
        #print
        lst2 = winpct_lst[start:end]
        lst2 = [x for x in lst2]

        print("Longest Streak at 70-win pace from " + str(lst1[0]) + " to " + str(lst1[-1]) + " | (" + str(max_count) + " games)")
        numgames = math.floor(((max(lst2)) * total_games))
        ##print("Highest Win % during Streak: " + str(round(100*max(lst2),2)) + "(" + str(numgames) + " games won)")
        numgames = math.floor((statistics.median(lst2)) * total_games)
        ##print("Median Win % during Streak: " + str(round(100*statistics.median(lst2),2)) + "(" + str(numgames) + " games won)")
        numgames = math.floor((min(lst2)) * total_games)
        ##print("Lowest Win % during Streak: " + str(round(100*min(lst2),2)) + "(" + str(numgames) + " games won)")
        ##print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        return_value = max_count 
        
    except AttributeError:
        print("URL: " + str(url))
        print(soup.find('div', {'id':'meta'}))
        print(soup.find('div', {'data-template':'Partials/Teams/Summary'}))
        #print(soup.find_all('span'))
        return_value = -1       
    return return_value

In [66]:
df = pd.read_csv('every_team_season.csv')

In [67]:
df = df.drop(columns=['Unnamed: 0'])

In [68]:
url_lst = list(df['URL'])
season_lst = list(df['Season'])

In [69]:
lst = []
for season in season_lst:
    if str(season)[-2:] != "00":
        lst.append(int(str(season)[0:2] + str(season)[-2:]))
    else:
        lst.append(2000)

In [70]:
season_lst = lst

In [71]:
lst = []
for i in range(len(url_lst)):
    if 'OKC' in url_lst[i] and season_lst[i] < 2009:
        url_lst[i] = 'https://www.basketball-reference.com/teams/SEA/'
    if 'NJN' in url_lst[i] and season_lst[i] > 2012:
        url_lst[i] = 'https://www.basketball-reference.com/teams/BRK/'
    if 'WAS' in url_lst[i] and season_lst[i] < 1998:
        url_lst[i] = 'https://www.basketball-reference.com/teams/WSB/'
    if 'CHA' in url_lst[i] and season_lst[i] < 2005:
        url_lst[i] = 'https://www.basketball-reference.com/teams/CHH/'
    if 'CHA' in url_lst[i] and season_lst[i] > 2014:
        url_lst[i] = 'https://www.basketball-reference.com/teams/CHO/'
    if 'NOH' in url_lst[i] and season_lst[i] > 2013:
        url_lst[i] = 'https://www.basketball-reference.com/teams/NOP/'
    if 'SAC' in url_lst[i] and season_lst[i] < 1986:
        url_lst[i] = 'https://www.basketball-reference.com/teams/KCK/'
    url = str(url_lst[i]) + str(season_lst[i]) + ".html"
    lst.append(url)

In [72]:
url_lst = lst

In [73]:
df = df.drop(columns=['Season','URL'])
df

,Lg,Team,W/L%,SRS,Rel Pace,ORtg,Rel ORtg,DRtg,Rel DRtg
0,NBA,Boston Celtics*,0.622,7.02,-1.6,114.4,2.4,106.9,-5.1
1,NBA,Boston Celtics*,0.500,1.32,-0.9,114.0,1.7,112.5,0.2
2,NBA,Boston Celtics*,0.667,5.83,-0.8,113.3,2.7,107.0,-3.6
3,NBA,Boston Celtics*,0.598,3.90,-0.4,112.2,1.8,107.8,-2.6
4,NBA,Boston Celtics*,0.671,3.23,-1.3,107.6,-1.0,103.9,-4.7
...,...,...,...,...,...,...,...,...,...
1598,NBA,Phoenix Suns,0.463,-0.96,2.3,98.4,1.6,99.5,2.7
1599,NBA,Phoenix Suns,0.598,5.57,1.7,101.3,3.4,96.5,-1.4
1600,NBA,Phoenix Suns,0.585,2.33,0.2,98.2,1.0,96.6,-0.6
1601,NBA,Phoenix Suns*,0.476,-1.66,-0.4,101.6,2.6,103.1,4.1


In [74]:
df['URL'] = url_lst
df['Season'] = season_lst
df

,Lg,Team,W/L%,SRS,Rel Pace,ORtg,Rel ORtg,DRtg,Rel DRtg,URL,Season
0,NBA,Boston Celtics*,0.622,7.02,-1.6,114.4,2.4,106.9,-5.1,https://www.basketball-reference.com/teams/BOS...,2022
1,NBA,Boston Celtics*,0.500,1.32,-0.9,114.0,1.7,112.5,0.2,https://www.basketball-reference.com/teams/BOS...,2021
2,NBA,Boston Celtics*,0.667,5.83,-0.8,113.3,2.7,107.0,-3.6,https://www.basketball-reference.com/teams/BOS...,2020
3,NBA,Boston Celtics*,0.598,3.90,-0.4,112.2,1.8,107.8,-2.6,https://www.basketball-reference.com/teams/BOS...,2019
4,NBA,Boston Celtics*,0.671,3.23,-1.3,107.6,-1.0,103.9,-4.7,https://www.basketball-reference.com/teams/BOS...,2018
...,...,...,...,...,...,...,...,...,...,...,...
1598,NBA,Phoenix Suns,0.463,-0.96,2.3,98.4,1.6,99.5,2.7,https://www.basketball-reference.com/teams/PHO...,1973
1599,NBA,Phoenix Suns,0.598,5.57,1.7,101.3,3.4,96.5,-1.4,https://www.basketball-reference.com/teams/PHO...,1972
1600,NBA,Phoenix Suns,0.585,2.33,0.2,98.2,1.0,96.6,-0.6,https://www.basketball-reference.com/teams/PHO...,1971
1601,NBA,Phoenix Suns*,0.476,-1.66,-0.4,101.6,2.6,103.1,4.1,https://www.basketball-reference.com/teams/PHO...,1970


In [80]:
temp = df.loc[df['SRS'] > 4]
temp = temp.loc[temp['Season'] > 1978] 
temp

,Lg,Team,W/L%,SRS,Rel Pace,ORtg,Rel ORtg,DRtg,Rel DRtg,URL,Season
0,NBA,Boston Celtics*,0.622,7.02,-1.6,114.4,2.4,106.9,-5.1,https://www.basketball-reference.com/teams/BOS...,2022
2,NBA,Boston Celtics*,0.667,5.83,-0.8,113.3,2.7,107.0,-3.6,https://www.basketball-reference.com/teams/BOS...,2020
11,NBA,Boston Celtics*,0.683,4.83,-1.7,106.2,-1.1,100.3,-7.0,https://www.basketball-reference.com/teams/BOS...,2011
13,NBA,Boston Celtics*,0.756,7.44,-1.3,110.5,2.2,102.3,-6.0,https://www.basketball-reference.com/teams/BOS...,2009
14,NBA,Boston Celtics*,0.805,9.30,-1.5,110.2,2.7,98.9,-8.6,https://www.basketball-reference.com/teams/BOS...,2008
...,...,...,...,...,...,...,...,...,...,...,...
1580,NBA,Phoenix Suns*,0.671,6.49,3.2,112.6,4.7,106.1,-1.8,https://www.basketball-reference.com/teams/PHO...,1991
1581,NBA,Phoenix Suns*,0.659,7.09,2.4,113.1,5.0,106.2,-1.9,https://www.basketball-reference.com/teams/PHO...,1990
1582,NBA,Phoenix Suns*,0.671,6.84,3.9,113.1,5.3,105.7,-2.1,https://www.basketball-reference.com/teams/PHO...,1989
1588,NBA,Phoenix Suns*,0.646,4.61,-1.5,105.1,0.4,100.1,-4.6,https://www.basketball-reference.com/teams/PHO...,1983


In [81]:
urls = list(temp['URL'])
#urls

In [82]:
lst = []
for url in urls:
    #print(url)
    lst.append((longest_streak(url), url))

2021-22 Boston Celtics 
Longest Streak at 70-win pace from Oct 20 to Apr 10 | (0 games)
2019-20 Boston Celtics 
Longest Streak at 70-win pace from Nov 7 to Nov 15 | (5 games)
2010-11 Boston Celtics 
Longest Streak at 70-win pace from Oct 26 to Oct 26 | (1 games)
2008-09 Boston Celtics 
Longest Streak at 70-win pace from Nov 21 to Dec 28 | (19 games)
2007-08 Boston Celtics 
Longest Streak at 70-win pace from Nov 29 to Jan 12 | (22 games)
1990-91 Boston Celtics 
Longest Streak at 70-win pace from Nov 2 to Nov 6 | (3 games)
1987-88 Boston Celtics 
Longest Streak at 70-win pace from Nov 6 to Nov 18 | (8 games)
1986-87 Boston Celtics 
Longest Streak at 70-win pace from Oct 31 to Oct 31 | (1 games)
1985-86 Boston Celtics 
Longest Streak at 70-win pace from Nov 26 to Dec 4 | (6 games)
1984-85 Boston Celtics 
Longest Streak at 70-win pace from Nov 12 to Dec 18 | (19 games)
1983-84 Boston Celtics 
Longest Streak at 70-win pace from Nov 9 to Nov 15 | (4 games)
1982-83 Boston Celtics 
Longest Str

In [83]:
lst.sort(reverse=True)
lst

[(81, 'https://www.basketball-reference.com/teams/GSW/2016.html'),
 (68, 'https://www.basketball-reference.com/teams/CHI/1996.html'),
 (61, 'https://www.basketball-reference.com/teams/CHI/1997.html'),
 (31, 'https://www.basketball-reference.com/teams/POR/1991.html'),
 (31, 'https://www.basketball-reference.com/teams/HOU/1994.html'),
 (30, 'https://www.basketball-reference.com/teams/SEA/1994.html'),
 (25, 'https://www.basketball-reference.com/teams/DAL/2003.html'),
 (22, 'https://www.basketball-reference.com/teams/MIL/2020.html'),
 (22, 'https://www.basketball-reference.com/teams/BOS/2008.html'),
 (19, 'https://www.basketball-reference.com/teams/LAL/2009.html'),
 (19, 'https://www.basketball-reference.com/teams/BOS/2009.html'),
 (19, 'https://www.basketball-reference.com/teams/BOS/1985.html'),
 (18, 'https://www.basketball-reference.com/teams/PHI/1983.html'),
 (18, 'https://www.basketball-reference.com/teams/PHI/1981.html'),
 (18, 'https://www.basketball-reference.com/teams/LAL/2002.htm